In [1]:
import os
os.makedirs('/root/.kaggle/', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [2]:
# Download the dataset
!kaggle datasets download -d arashnic/book-recommendation-dataset

# Unzip the specific CSV file
!unzip book-recommendation-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset
License(s): CC0-1.0
 53% 13.0M/24.3M [00:00<00:00, 66.0MB/s]
100% 24.3M/24.3M [00:00<00:00, 102MB/s] 
Archive:  book-recommendation-dataset.zip
  inflating: Books.csv               
  inflating: DeepRec.png             
  inflating: Ratings.csv             
  inflating: Users.csv               
  inflating: classicRec.png          
  inflating: recsys_taxonomy2.png    


In [3]:
import numpy as np
import pandas as pd

In [4]:
books = pd.read_csv('Books.csv')
ratings = pd.read_csv('Ratings.csv')
users = pd.read_csv('Users.csv')

<ipython-input-4-f6677f1956ea>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


In [5]:
# drop all rows with na in books
books.dropna(inplace=True)

In [6]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [7]:
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [8]:
ratings.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [9]:
books['Image-URL-S'] = books['Image-URL-S'].str.replace('http://', 'https://')
books['Image-URL-M'] = books['Image-URL-M'].str.replace('http://', 'https://')
books['Image-URL-L'] = books['Image-URL-L'].str.replace('http://', 'https://')

In [10]:
ratingsmerge = ratings.merge(books, on='ISBN')

## Popularity Based Reccomendation System

In [11]:
numrating_df=ratingsmerge.groupby('Book-Title').count()['Book-Rating'].reset_index()
numrating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
numrating_df

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241060,Ã?Â?lpiraten.,2
241061,Ã?Â?rger mit Produkt X. Roman.,4
241062,Ã?Â?sterlich leben.,1
241063,Ã?Â?stlich der Berge.,3


In [12]:
avgrating_df=ratingsmerge.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avgrating_df.rename(columns={'Book-Rating':'avg_rating'},inplace=True)
avgrating_df

,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241060,Ã?Â?lpiraten.,0.000000
241061,Ã?Â?rger mit Produkt X. Roman.,5.250000
241062,Ã?Â?sterlich leben.,7.000000
241063,Ã?Â?stlich der Berge.,2.666667


In [13]:
popular_df_all=numrating_df.merge(avgrating_df,on='Book-Title')
popular_df_all

,Book-Title,num_ratings,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241060,Ã?Â?lpiraten.,2,0.000000
241061,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241062,Ã?Â?sterlich leben.,1,7.000000
241063,Ã?Â?stlich der Berge.,3,2.666667


In [39]:
popular_df_all['product'] = popular_df_all['avg_rating'] * popular_df_all['num_ratings']

# Sort the dataframe by the 'product' column in descending order
popular_df_all_sorted = popular_df_all.sort_values(by='product', ascending=False)

# Drop num and avg
popular_df_all_sorted = popular_df_all_sorted.drop(columns=['num_ratings', 'avg_rating'])

In [47]:
result = popular_df_all_sorted.join(books.set_index('Book-Title'), on='Book-Title')

# Select the desired columns
result = result[['Book-Title', 'product', 'Book-Author', 'Publisher']]

In [53]:
result = result.groupby('Book-Title').agg({
    'product': 'mean',
    'Book-Author': 'first',  # Use 'first' to keep the Book-Author for each group
    'Publisher': 'first'     # Use 'first' to keep the Publisher for each group
})

# Sort the grouped result by 'product' in descending order
result = result.sort_values(by='product', ascending=False).reset_index()

In [14]:
popular_df= popular_df_all[popular_df_all['num_ratings']>=250].sort_values('avg_rating',ascending=False)
popular_df

,Book-Title,num_ratings,avg_rating
80431,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
80419,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
80438,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
80423,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
80411,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
...,...,...,...
227695,Vinegar Hill (Oprah's Book Club (Paperback)),265,2.245283
233629,Whispers,286,2.199301
143373,Presumed Innocent,294,2.139456
94379,Isle of Dogs,288,2.000000


In [15]:
popular_df= popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']].head(48)

In [16]:
popular_df

,Book-Title,Book-Author,Image-URL-M,num_ratings,avg_rating
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,https://images.amazon.com/images/P/0439136350....,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,https://images.amazon.com/images/P/0439139597....,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,https://images.amazon.com/images/P/0590353403....,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,https://images.amazon.com/images/P/043935806X....,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,https://images.amazon.com/images/P/0439064872....,556,5.183453
16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,https://images.amazon.com/images/P/0345339681....,281,5.007117
17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,https://images.amazon.com/images/P/0345339703....,368,4.948370
26,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,https://images.amazon.com/images/P/059035342X....,575,4.895652
28,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,https://images.amazon.com/images/P/0345339711....,260,4.880769
39,To Kill a Mockingbird,Harper Lee,https://images.amazon.com/images/P/0446310786....,510,4.700000


## Collaborative Based Reccomendation System

In [17]:
x=ratingsmerge.groupby('User-ID').count()['Book-Rating']>200
experienced_users=x[x].index

In [18]:
filtered_rating= ratingsmerge[ratingsmerge['User-ID'].isin(experienced_users)]
filtered_rating

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,https://images.amazon.com/images/P/034545104X....,https://images.amazon.com/images/P/034545104X....,https://images.amazon.com/images/P/034545104X....
5,23768,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,https://images.amazon.com/images/P/034545104X....,https://images.amazon.com/images/P/034545104X....,https://images.amazon.com/images/P/034545104X....
7,28523,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,https://images.amazon.com/images/P/034545104X....,https://images.amazon.com/images/P/034545104X....,https://images.amazon.com/images/P/034545104X....
15,77940,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,https://images.amazon.com/images/P/034545104X....,https://images.amazon.com/images/P/034545104X....,https://images.amazon.com/images/P/034545104X....
16,81977,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,https://images.amazon.com/images/P/034545104X....,https://images.amazon.com/images/P/034545104X....,https://images.amazon.com/images/P/034545104X....
...,...,...,...,...,...,...,...,...,...,...
1030875,275970,1880837927,0,The Theology of the Hammer,Millard Fuller,1994,Smyth &amp; Helwys Publishing,https://images.amazon.com/images/P/1880837927....,https://images.amazon.com/images/P/1880837927....,https://images.amazon.com/images/P/1880837927....
1030876,275970,188717897X,0,The Ordeal of Integration: Progress and Resent...,Orlando Patterson,1998,Civitas Book Publisher,https://images.amazon.com/images/P/188717897X....,https://images.amazon.com/images/P/188717897X....,https://images.amazon.com/images/P/188717897X....
1030877,275970,1888889047,0,Pushcart's Complete Rotten Reviews &amp; Rejec...,Bill Henderson,1998,Pushcart Press,https://images.amazon.com/images/P/1888889047....,https://images.amazon.com/images/P/1888889047....,https://images.amazon.com/images/P/1888889047....
1030878,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),https://images.amazon.com/images/P/1931868123....,https://images.amazon.com/images/P/1931868123....,https://images.amazon.com/images/P/1931868123....


In [19]:
y=filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books=y[y].index

In [20]:
famous_books

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Day Late and a Dollar Short', 'A Fine Balance',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Book-Title', length=706)

In [21]:
final_ratings= filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]
final_ratings

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
63,278418,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,https://images.amazon.com/images/P/0446520802....,https://images.amazon.com/images/P/0446520802....,https://images.amazon.com/images/P/0446520802....
65,3363,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,https://images.amazon.com/images/P/0446520802....,https://images.amazon.com/images/P/0446520802....,https://images.amazon.com/images/P/0446520802....
66,7158,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,https://images.amazon.com/images/P/0446520802....,https://images.amazon.com/images/P/0446520802....,https://images.amazon.com/images/P/0446520802....
69,11676,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,https://images.amazon.com/images/P/0446520802....,https://images.amazon.com/images/P/0446520802....,https://images.amazon.com/images/P/0446520802....
74,23768,0446520802,6,The Notebook,Nicholas Sparks,1996,Warner Books,https://images.amazon.com/images/P/0446520802....,https://images.amazon.com/images/P/0446520802....,https://images.amazon.com/images/P/0446520802....
...,...,...,...,...,...,...,...,...,...,...
1026716,266865,0531001725,10,The Catcher in the Rye,Jerome David Salinger,1973,Scholastic Library Pub,https://images.amazon.com/images/P/0531001725....,https://images.amazon.com/images/P/0531001725....,https://images.amazon.com/images/P/0531001725....
1027915,269566,0670809381,0,Echoes,Maeve Binchy,1986,Penguin USA,https://images.amazon.com/images/P/0670809381....,https://images.amazon.com/images/P/0670809381....,https://images.amazon.com/images/P/0670809381....
1028769,271284,0440910927,0,The Rainmaker,John Grisham,1995,Island,https://images.amazon.com/images/P/0440910927....,https://images.amazon.com/images/P/0440910927....,https://images.amazon.com/images/P/0440910927....
1029062,271705,B0001PIOX4,0,Fahrenheit 451,Ray Bradbury,1993,Simon &amp; Schuster,https://images.amazon.com/images/P/B0001PIOX4....,https://images.amazon.com/images/P/B0001PIOX4....,https://images.amazon.com/images/P/B0001PIOX4....


In [22]:
pt=final_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')

In [23]:
pt.fillna(0,inplace=True)

In [24]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
similarity_score= cosine_similarity(pt)

In [27]:
books.isna().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Image-URL-S            0
Image-URL-M            0
Image-URL-L            0
dtype: int64

In [29]:
import difflib

In [121]:
def get_recommendations_by_criteria(criteria_value, column_name, exclude_book, result):
    criteria_books = result[result[column_name] == criteria_value]['Book-Title'].tolist()
    criteria_books = [book for book in criteria_books if book != exclude_book]
    return criteria_books[:6]

def recommend(book_name, pt, result, similarity_score):
    # Step 1: Check for partial matches in the pivot table
    partial_matches_pt = pt.index[pt.index.str.contains(book_name, case=False, na=False, regex=False)].tolist()
    if partial_matches_pt:
        # If partial matches are found in the pivot table
        best_match = partial_matches_pt[0]
        index = np.where(pt.index == best_match)[0][0]
        similar_items = sorted(list(enumerate(similarity_score[index])), key=lambda x: x[1], reverse=True)

        print(f"Books containing '{book_name}' in the pivot table:")
        for title in partial_matches_pt[:5]:  # Limiting to 5 partial match results
            print(title)

        print(f"\nRecommendations based on the book '{best_match}':")
        count = 0
        for i in similar_items:
            if pt.index[i[0]] not in partial_matches_pt:
                print(pt.index[i[0]])
                count += 1
                if count == 8:  # Limiting to 8 similar recommendations
                    break
    else:
        # Step 2: Check for partial matches in the result DataFrame
        partial_matches_result = result[result['Book-Title'].str.contains(book_name, case=False, na=False, regex=False)]['Book-Title'].unique().tolist()
        close_matches_result = difflib.get_close_matches(book_name, result['Book-Title'], n=1, cutoff=0.6)

        if partial_matches_result:
            print(f"Books containing '{book_name}' in the result DataFrame:")
            for title in partial_matches_result[:10]:  # Limiting to 10 results
                print(title)

        if close_matches_result:
            best_match_book = close_matches_result[0]
            book_details = result[result['Book-Title'] == best_match_book].iloc[0]
            author = book_details['Book-Author']
            publisher = book_details['Publisher']

            # Recommend books by the same author and publisher
            print(f"\nShowing recommendations based on the book '{best_match_book}' by the same author '{author}' and publisher '{publisher}':")
            print("Books by the same author:")
            author_recommendations = get_recommendations_by_criteria(author, 'Book-Author', best_match_book, result)
            for book in author_recommendations[:6]:  # Limiting to 6 recommendations from author
                print(book)

            print("\nBooks from the same publisher:")
            publisher_recommendations = get_recommendations_by_criteria(publisher, 'Publisher', best_match_book, result)
            for book in publisher_recommendations[:6]:  # Limiting to 6 recommendations from publisher
                print(book)
        else:
            print("No similar book found in the dataset.")


In [125]:
recommend('After', pt, result, similarity_score)

Books containing 'After' in the pivot table:
Long After Midnight

Recommendations based on the book 'Long After Midnight':
Final Target
Waiting to Exhale
River's End
Last Man Standing
Presumed Innocent
Saving Faith
Inner Harbor (Quinn Brothers (Paperback))
Engaging The Enemy
